# Get and examine data

## Download data

In [ ]:
# Necessary data science packages
import sys
from packaging import version
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np

wine_dataset = sklearn.datasets.load_wine()
wine_df = pd.DataFrame(data=wine_dataset.data, columns=wine_dataset.feature_names)
wine_df['labels'] = wine_dataset['target'] # Also add the labels associated with each sample

X, y = wine_dataset.data, wine_dataset.target

In [ ]:
X.shape

In [ ]:
y

In [ ]:
y.shape

## Inspect the data

In [ ]:
# Display the head of the wine dataframe. 
wine_df.head()

In [ ]:
# Display column names and types
wine_df.info()

In [ ]:
# Get the value counts for each different type of wine. 
wine_df['labels'].value_counts()

It seems that the labels are roughly balanced, although wine type #1 is the most common. Now, let's do a train-test split. 

In [ ]:
def shuffle_and_split_data(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = shuffle_and_split_data(wine_df, 0.2)
len(train_set)

In [ ]:
# Allows results of the notebook to be reproducible. 
np.random.seed(42)

In [ ]:
len(test_set)

# Training and Predictions

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train.shape
X_test.shape
y_train.shape
y_test.shape

We have successfully performed a train-test split with 143 samples in the training set and 35 samples in the test set.
Since we are dealing with a small dataset for multi-class classification, it might be helpful to use the SVM classifier. 

In [ ]:
from sklearn.svm import SVC

# Instantiate and fit on the training set. 
svm_clf = SVC(random_state=42)
svm_clf.fit(X_train, y_train)

In [ ]:
# Predict on the testing set. 
predictions = svm_clf.predict(X_test)

In [ ]:
# Now, let's evaluate the accuracy of the predictions. 
from sklearn.metrics import accuracy_score as accuracy
round(accuracy(y_test, predictions), 2)

We get an accuracy of around 71%. Let's plot our results in a confusion matrix. 

In [ ]:
from sklearn import metrics
from sklearn.metrics import ConfusionMatrixDisplay

conf_matrix = metrics.confusion_matrix(y_test, predictions)
cm = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0,1,2])
cm.plot();

There are quite a few misclassified samples. Let's try using a different model to see if we get better results. We can use a Decision Tree. To avoid overfitting, let's set the max_depth to be 2. 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

Now, let's visualize the resulting dendrogram. 

In [ ]:
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
from pathlib import Path

#Define the class labels. 
class_labels_str = [str(elem) for elem in set(y)]
wine_classes = np.array(class_labels_str)

plt.figure(figsize=(12,12))


dendro = sklearn.tree.plot_tree(tree_clf, filled=True, rounded=True, feature_names=wine_dataset.feature_names, 
                                class_names=wine_classes, fontsize=12)

plt.show()

From the dendrogram, we can see that there are less misclassifications than the support vector machine model. However, while the SVC model had 0 misclassifications for class 0, the DecisionTree misclassifies two samples as belonging to class 0. 

# Additional Visualizations

Let's now visualize wine samples based on the features picked out by the decision tree. We can indicate the true labels of the samples on the plot. 

In [ ]:
fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(wine_df['proline'], wine_df['od280/od315_of_diluted_wines'], wine_df['flavanoids'], marker='o', c=wine_df['labels'])
ax.set_xlabel('proline')
ax.set_ylabel('od280/od315')
ax.set_zlabel('flavanoids')

# Add a legend
cbar = fig.colorbar(scatter, label="Class", orientation='horizontal')
cbar.set_ticks([0, 1, 2])
plt.show()

As we can see, it appears that these three features are good for distinguishing between the different wine classes. For the most part, the yellow, blue, and purple points don't overlap, although there are a few exceptions. 

# Attributions

1. https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html
2. https://scikit-learn.org/stable/modules/svm.html
3. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html
4. https://www.w3schools.com/python/python_ml_confusion_matrix.asp
5. https://scikit-learn.org/stable/modules/classes.html#module-sklearn.tree
6. https://stackoverflow.com/questions/59447378/sklearn-plot-tree-plot-is-too-small
7. https://stackoverflow.com/questions/6715442/how-to-add-matplotlib-colorbar-ticks
8. https://www.kaggle.com/code/cristianlapenta/wine-dataset-sklearn-machine-learning-project
